In [1]:
# Plunkylib creates its files in the current working directory, so let's change to the parent directory if we're in the notebooks directory, since we want to use the default samples
import os
if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")

# we also want to ensure we have a .env file that contains our API keys, so warn if we don't have that file present in this directory
if not os.path.exists(".env"):
    print("WARNING: No .env file found in this directory. Please create one with the API Key contents mentioned in the README.md file.")

In [2]:
from plunkylib import *
import dataclasses


In [ ]:
# with datafiles, you can look up an existing datafile object by name

# lets mimic the readme but using the objects instead of the CLI
example_params_to_copy = CompletionParams.objects.get("ExampleGPT3")
print(example_params_to_copy)

# make a copy and replace the name
params_myfirstengine = dataclasses.replace(example_params_to_copy, name="MyFirstEngine")
print(params_myfirstengine)

In [ ]:
# Bonus: If you wanted, you can change the parameters and the file will change accordingly
params_myfirstengine.max_tokens = 100
print(params_myfirstengine)

In [ ]:
# same thing with the prompt
example_prompt_to_copy = Prompt.objects.get("ExampleSimple")
print(example_prompt_to_copy)

# make a copy and replace the name
prompt_myfirstprompt = dataclasses.replace(example_prompt_to_copy, name="MyFirstPrompt")
prompt_myfirstprompt

In [ ]:
# now a new Petition using the names of the prompt and params
petition_example = Petition("MyFirstPetition", prompt_myfirstprompt.name, params_myfirstengine.name)
petition_example

In [ ]:
# petitions have some lazy references that we tend to load right after if we want to complete with them
petition_example.load_all()

example_completion, adj_prompt_text = await petition_completion2(petition=petition_example, additional={}, content_filter_check=True)

In [ ]:
# Completion object contains our results
print(example_completion.text)
print(example_completion.content_filter_rating)

In [ ]:
# We can change the prompt, if we want
prompt_myfirstprompt.text = "Describe, in great detail, the smell of the beach after an oil spill.\n\nDESCRIPTION:\n"

In [ ]:
#And you don't need to rebuild a new petition, the original one works great
#You keep reusing it, and you can edit the prompt (or engine parameters) from code or from the text/yaml files
example_completion2, adj_prompt_text = await petition_completion2(petition=petition_example, additional={}, content_filter_check=True)

# Completion object contains our results
print(example_completion2.text)
print(example_completion2.content_filter_rating)

In [ ]:
# We could run the same thing with Jurassic, we just need new CompletionParams and a new petition that uses our prompt
sample_jparams = CompletionParams.objects.get("ExampleJurassicJumbo")
params_mysecondengine = dataclasses.replace(sample_jparams, name="MySecondEngine")
params_mysecondengine.max_tokens = 100

# We don't need a new prompt, we can use both
petition_example2 = Petition("MySecondPetition", prompt_myfirstprompt.name, params_mysecondengine.name)
petition_example2

In [ ]:
params_mysecondengine.max_tokens = 100
example_completion3, adj_prompt_text = await petition_completion2(petition=petition_example2, additional={}, content_filter_check=True)

# Completion object contains our results
print(example_completion3.text)
print(example_completion3.content_filter_rating)

In [ ]:
# Test a chat prompt
petition_example3 = Petition.objects.get("ExampleChat")
petition_example3.load_all()
example_completion3, adj_prompt_text = await petition_completion2(petition=petition_example3, additional={}, content_filter_check=False)
print(example_completion3.text)